## Setup a classification experiment

In [3]:
import pickle
import numpy as np

In [47]:
m1 = [0, +1 , -1 , 5]
w1= [1,2,3]
edges1 =[1,50,100]

m2 = [0,+2, -2, +2 , +3 , 9]
w2 =[10,20,30,40,50]
edges2 =[20,33,66,90 , 120]

merged_edges = sorted(set().union(*[set(edges1) , set(edges2)]))
print("Merged Edges = \n" , merged_edges)

models =[ m1 , m2]
weights= [w1,w2]

ne = [ np.searchsorted( edges1,  merged_edges + [np.inf] ) , np.searchsorted(edges2,  merged_edges + [np.inf])]
# print('Edge Index= ', ne)

bv =[]
wv=[]
for idx, m in enumerate(models):
    mvalues = m[1:]    
    bv.append([ mvalues[x-1] if x > 0 and x <=len(mvalues) else 0 for x in ne[idx][1:] ])
    wv.append([ weights[idx][x-1] if x > 0 and x <=len(mvalues) else 0 for x in ne[idx][1:] ])

print("New Bin Values =")
for b in bv: print(len(b))

print("New Weight Values =")
for w in wv: print(len(w))

mbv = np.average(bv, axis=0, weights= wv)# np.nanmean(bv, axis=0)

print("Merged models bin values= \n" , mbv)

mbv = np.average(bv, axis=0)
# Merged Model=  [1.   1.5 -0.5 -1.5  0.5  1.   4.   7. ]

Merged Edges = 
 [1, 20, 33, 50, 66, 90, 100, 120]
New Bin Values =
8
8
New Weight Values =
8
8
Merged models bin values= 
 [ 1.          1.90909091 -1.85714286 -1.90909091  1.8125      2.80952381
  3.13953488  8.77358491]


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
# df = df.sample(frac=0.1, random_state=1)
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

## Explore the dataset

In [14]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
show(hist)

<!-- http://127.0.0.1:7001/2132562747144/ -->

## Train the Explainable Boosting Machine (EBM)

In [4]:
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier

ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

ExplainableBoostingClassifier(feature_names=['Age', 'WorkClass', 'fnlwgt',
                                             'Education', 'EducationNum',
                                             'MaritalStatus', 'Occupation',
                                             'Relationship', 'Race', 'Gender',
                                             'CapitalGain', 'CapitalLoss',
                                             'HoursPerWeek', 'NativeCountry',
                                             'Relationship x HoursPerWeek',
                                             'Age x Relationship',
                                             'MaritalStatus x HoursPerWeek',
                                             'EducationNum x Occupation',
                                             'fnlwgt x HoursPerWeek',
                                             'Occupat...
                              feature_types=['continuous', 'categorical',
                                             'co

## Mergring Multiple Explainable Boosting Machines (EBM)

In [4]:
from sklearn.model_selection import KFold
from interpret.glassbox import ExplainableBoostingClassifier

rng = np.random.RandomState(0)
cv = KFold(n_splits=3, shuffle=True, random_state=rng)
models =[]
seed =1
for train_index, test_index in cv.split(X):

    X_train, y_train = X.iloc[train_index], y.iloc[train_index]

    print( len(X_train) , len(y_train))
    
    ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
    ebm.fit(X_train, y_train)  
    models.append(ebm)
    seed +=10


21707 21707
21707 21707
21708 21708


In [5]:
len(models)

3

In [12]:
%load_ext autoreload
%autoreload 2
from interpret.glassbox.ebm.utils import *

merged_ebm = EBMUtils.merge_models(models=models)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
ebm0_global = models[0].explain_global(name='EBM')
show(ebm0_global)

<!-- http://127.0.0.1:7001/2132628760968/ -->

In [18]:
ebm1_global = models[1].explain_global(name='EBM')
show(ebm1_global)

<!-- http://127.0.0.1:7001/2132626967880/ -->

In [19]:
ebm2_global = models[2].explain_global(name='EBM')
show(ebm2_global)

<!-- http://127.0.0.1:7001/2132626793864/ -->

In [15]:
ebm_global = merged_ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7001/2132594776520/ -->

In [12]:
seed =1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
ebm1 = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
print( len(X_train) , len(y_train))
ebm1.fit(X_train, y_train)  

seed +=10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
print( len(X_train) , len(y_train))
ebm2 = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm2.fit(X_train, y_train)  

seed +=10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
print( len(X_train) , len(y_train))
ebm3 = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm3.fit(X_train, y_train)  

26048 26048
26048 26048
26048 26048


ExplainableBoostingClassifier(feature_names=['Age', 'WorkClass', 'fnlwgt',
                                             'Education', 'EducationNum',
                                             'MaritalStatus', 'Occupation',
                                             'Relationship', 'Race', 'Gender',
                                             'CapitalGain', 'CapitalLoss',
                                             'HoursPerWeek', 'NativeCountry',
                                             'Age x Relationship',
                                             'Relationship x HoursPerWeek',
                                             'Relationship x CapitalLoss',
                                             'WorkClass x Race',
                                             'Age x HoursPerWeek',
                                             'EducationNum x Occupati...
                              feature_types=['continuous', 'categorical',
                                             '

In [6]:
from interpret.glassbox.ebm.utils import *

merged_ebm = EBMUtils.merge_models(models=[ebm1, ebm2 , ebm3])

AttributeError: 'ExplainableBoostingClassifier' object has no attribute 'preprocessor_'

In [29]:
pickle.dump(ebm1,open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm1.pickle", 'wb'))
pickle.dump(ebm2,open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm2.pickle", 'wb'))
pickle.dump(ebm3,open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm3.pickle", 'wb'))

In [30]:
ebm1 = pickle.load(open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm1.pickle" ,'rb'))
ebm2 = pickle.load(open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm2.pickle" ,'rb'))
ebm3 = pickle.load(open("D:\\Projects\\interpret\\examples\\python\\notebooks\\ebm3.pickle" ,'rb'))


In [70]:
ebm2.feature_names

['Age',
 'WorkClass',
 'fnlwgt',
 'Education',
 'EducationNum',
 'MaritalStatus',
 'Occupation',
 'Relationship',
 'Race',
 'Gender',
 'CapitalGain',
 'CapitalLoss',
 'HoursPerWeek',
 'NativeCountry',
 'Relationship x HoursPerWeek',
 'Age x Relationship',
 'MaritalStatus x HoursPerWeek',
 'EducationNum x MaritalStatus',
 'Age x CapitalLoss',
 'Age x HoursPerWeek',
 'Age x WorkClass',
 'WorkClass x Race',
 'Relationship x CapitalLoss',
 'Occupation x Relationship']

In [76]:
ebm1.preprocessor_.col_bin_counts_[0] , ebm2.preprocessor_.col_bin_counts_[0]

(array([  0, 317, 438, 568, 606, 575, 613, 697, 641, 669, 617, 672, 699,
        657, 684, 736, 659, 703, 696, 702, 719, 688, 663, 656, 643, 660,
        621, 604, 579, 574, 595, 561, 430, 460, 485, 485, 374, 373, 321,
        338, 289, 282, 297, 288, 254, 241, 199, 191, 170, 133, 111, 127,
         97,  84,  75,  55,  52,  51,  39,  37,  36,  25,  19,  37,  25,
         21,  35], dtype=int64),
 array([  0, 303, 445, 558, 607, 579, 618, 683, 644, 676, 606, 665, 705,
        639, 700, 704, 652, 714, 701, 708, 717, 677, 645, 640, 640, 630,
        631, 615, 583, 586, 599, 578, 434, 480, 492, 483, 384, 372, 340,
        333, 303, 286, 293, 294, 243, 235, 212, 183, 175, 144, 117, 117,
        102,  91,  72,  62,  48,  52,  31,  32,  35,  22,  20,  35,  23,
         18,  37], dtype=int64))

In [95]:
(ebm1.preprocessor_.col_bin_counts_[0])

array([  0, 317, 438, 568, 606, 575, 613, 697, 641, 669, 617, 672, 699,
       657, 684, 736, 659, 703, 696, 702, 719, 688, 663, 656, 643, 660,
       621, 604, 579, 574, 595, 561, 430, 460, 485, 485, 374, 373, 321,
       338, 289, 282, 297, 288, 254, 241, 199, 191, 170, 133, 111, 127,
        97,  84,  75,  55,  52,  51,  39,  37,  36,  25,  19,  37,  25,
        21,  35], dtype=int64)

In [133]:
for index,_ in enumerate(ebm1.feature_groups_):
    if index in ebm1.preprocessor_.col_bin_edges_.keys():
        print(index, len(ebm1.preprocessor_.col_bin_edges_[index]))

0 65
2 254
4 15
10 75
11 58
12 76


In [138]:
from interpret.glassbox.ebm.utils import *

In [ ]:
EBMUtils.merge_models(models)

## Global Explanations: What the model learned overall

In [5]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7001/2793533953672/ -->

## Local Explanations: How an individual prediction was made

In [ ]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5], name='EBM')
show(ebm_local)

## Evaluate EBM performance

In [ ]:
from interpret.perf import ROC

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

## Let's test out a few other Explainable Models

In [ ]:
from interpret.glassbox import LogisticRegression, ClassificationTree

# We have to transform categorical variables to use Logistic Regression and Decision Tree
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)
X_train_enc, X_test_enc, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)

lr = LogisticRegression(random_state=seed, feature_names=feature_names, penalty='l1', solver='liblinear')
lr.fit(X_train_enc, y_train)

tree = ClassificationTree()
tree.fit(X_train_enc, y_train)

## Compare performance using the Dashboard

In [ ]:
lr_perf = ROC(lr.predict_proba).explain_perf(X_test_enc, y_test, name='Logistic Regression')
tree_perf = ROC(tree.predict_proba).explain_perf(X_test_enc, y_test, name='Classification Tree')

show(lr_perf)
show(tree_perf)
show(ebm_perf)

## Glassbox: All of our models have global and local explanations

In [ ]:
lr_global = lr.explain_global(name='Logistic Regression')
tree_global = tree.explain_global(name='Classification Tree')

show(lr_global)
show(tree_global)
show(ebm_global)

## Dashboard: look at everything at once

In [ ]:
# Do everything in one shot with the InterpretML Dashboard by passing a list into show

show([hist, lr_global, lr_perf, tree_global, tree_perf, ebm_global, ebm_perf], share_tables=True)